# Change long names
This notebook is designed to modify the `long_name` attribute of the variables, using the `altLabel` from the P35 Vocabulary.
In addition the `long_name` is modified so that the variable name appears first.

## Examples
```bash
Water body dissolved inorganic nitrogen -> Inorganic nitrogen_dissolved 
Deepest depth for Water body dissolved oxygen concentration → Oxygen deepest depth
```

## Input
The notebook will loop over the netCDF files located inside the directory `datadir`.

In [1]:
using NCDatasets
using Glob
using DIVAnd

## Generate list of files

In [71]:
datadir = "/home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/ogs04/All_European_Seas-water_body/"
datadir = "/home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/Coastal_areas-water_body/Baltic_Sea_-_Gulf_of_Riga/"
datadir = "/home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Baltic_Sea/"
datafilelist = glob("*nc", datadir)

5-element Vector{String}:
 "/home/ctroupin/data/EMODnet-Che" ⋯ 73 bytes ⋯ "ter_body_chlorophyll-a.4Danl.nc"
 "/home/ctroupin/data/EMODnet-Che" ⋯ 88 bytes ⋯ "ved_inorganic_nitrogen.4Danl.nc"
 "/home/ctroupin/data/EMODnet-Che" ⋯ 90 bytes ⋯ "d_oxygen_concentration.4Danl.nc"
 "/home/ctroupin/data/EMODnet-Che" ⋯ 69 bytes ⋯ "a/Water_body_phosphate.4Danl.nc"
 "/home/ctroupin/data/EMODnet-Che" ⋯ 68 bytes ⋯ "ea/Water_body_silicate.4Danl.nc"

In [72]:
for datafile in datafilelist
    @info("Working on file $(datafile)")
    NCDataset(datafile, "a") do ds
        varlist = keys(ds)

        # Identify the main variable name
        # mainvar_ = first(varlist)
        mainvar_ = first(split(last(keys(ds)), "_"))
        mainvarshort = replace(mainvar_, "Water_body_" => "", "Water body " => "", "_" => " ")
        @info("Main variable: $(mainvar_), $(mainvarshort)")

        # Get the parameter URN
        parameter_keyword_urn = ds.attrib["parameter_keyword_urn"]
        @info(parameter_keyword_urn)

        # Generate alternative label using Vocab
        newlabel = Vocab.altLabel(parameter_keyword_urn)
        @info("The new label is $(newlabel)")
        @info("")

        # Loop on variables
        for varname in varlist
            @debug(varname)
            
            # Work only on variables related to parameter (not coordinates etc)
            if startswith(varname, mainvar_)

                varlongname = ds[varname].attrib["long_name"]
                @debug("Long name: $(varlongname)")

                # replace "for" with "of"
                newvarlongname = replace(varlongname, "for " => "of ", "_" => " ")
                newvarlongname = replace(newvarlongname, "Water body " => "")
                @debug(newvarlongname)

                namesplit = split(newvarlongname, mainvarshort)
                if namesplit[end] == ""
                    newvarlongname = "$(newlabel) $(replace(first(namesplit), " of " => ""))"
                else
                    newvarlongname = "$(newlabel) $(replace(first(namesplit), " of " => "")) $(last(namesplit))"
                end
                newvarlongname = replace(newvarlongname, "   " => " ", "  " => " ", "(DIN) " => "")
                newvarlongname = uppercasefirst(lowercase(newvarlongname))

                @info("$(varlongname) → $(newvarlongname)")

                # Perform change
                ds[varname].attrib["long_name"] = rstrip(newvarlongname)
            end
        end
    end
end

[ Info: Working on file /home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Baltic_Sea/Water_body_chlorophyll-a.4Danl.nc
[ Info: Main variable: Water body chlorophyll-a, chlorophyll-a
[ Info: SDN:P35::EPC00105
[ Info: The new label is Chlorophyll-a_particulate
[ Info: 
[ Info: Water body chlorophyll-a → Chlorophyll-a_particulate 
[ Info: Water body chlorophyll-a masked using relative error threshold 0.3 → Chlorophyll-a_particulate masked using relative error threshold 0.3
[ Info: Water body chlorophyll-a masked using relative error threshold 0.5 → Chlorophyll-a_particulate masked using relative error threshold 0.5
[ Info: Relative error of Water body chlorophyll-a → Chlorophyll-a_particulate relative error
[ Info: Deepest values of Water body chlorophyll-a → Chlorophyll-a_particulate deepest values
[ Info: Deepest values of Water body chlorophyll-a masked using relative error threshold 0.3 → Chlorophyll-a_particulate deepest values masked using re